In [1]:
import sys, re
import pprint

# https://github.com/fzinfz/meter-dlt645
sys.path.append("/data/powermeter_dlt645/meter-dlt645")

from test_dlt645 import *
import test_dlt645

def read_chn(addr, cmd, verbose = 0):
    chn.encode(addr, 0x11, cmd)
    chn.xchg_data(verbose)
    return chn.rx_payload

In [2]:
verbose=0

serial_port = '/dev/ttyUSB0'
timeout_count = 10
wait_for_read = 0.5

chn=dlt645.Channel(port_id = serial_port, tmo_cnt = timeout_count, wait_for_read = wait_for_read)

In [3]:
#                        （DI3~0, 整数位数，小数位数， 单位）
D = {
    '当前组合有功总电能':       ('00 00 00 00', 6, 2, 'kWh'),
    '上月组合有功总电能':       ('00 00 00 01', 6, 2, 'kWh'),
    
    'A相电压':             ('02 01 01 00', 3, 1, 'V'),    
    'A相电流':             ('02 02 01 00', 3, 3, 'A'),    
    '瞬时总有功功率':       ('02 03 00 00', 2, 4, 'kW'),    

    '总功率因数':           ('02 06 00 00', 1, 3, ''),  
    
}

def get_data(addr, item):
    d = D[item]    
    payload = read_chn( addr, [ int(x) for x in d[0].split(' ')[::-1] ] )
    
    len_whole, len_decimal = d[1], d[2]
    len_payload = int( (len_whole + len_decimal) / 2 )
    return int(''.join([ "%02x" % x  for x in payload[::-1][:len_payload] ])) / pow(10, len_decimal)


def read_meter(addr):
    
    result = {}

    for item in D:
        result[item] = get_data(addr, item)

    result['A相电压x电流'] = "{:,.2f}".format( result['A相电压'] * result['A相电流'] )
    result['当前-上月'] = "{:,.2f}".format( result['当前组合有功总电能'] - result['上月组合有功总电能'] )
    return result

not working: 

    '瞬时总视在功率':       ('02 05 00 00', 2, 4, 'kVA'),    
    '零线电流':             ('02 80 00 00', 1, 3, 'A'),  
    '一分钟有功总平均功率':   ('02 80 00 03', 2, 4, 'kW'),          

In [4]:
s = '''
000002005433
000102866265
'''

chn.open()

result = {}

for addr_human in s.strip().splitlines():
    
    print('\n', '='* 20, addr_human, '='* 20)
    addr = [ int(s,16) for s in re.findall('..',addr_human) ]
    rs = read_meter(addr)
    pprint.pprint(rs)
    result[addr_human] = rs


 ==================== 000002005433 ====================
{'A相电压': 239.8,
 'A相电压x电流': '90.40',
 'A相电流': 0.377,
 '上月组合有功总电能': 779.63,
 '当前-上月': '134.78',
 '当前组合有功总电能': 914.41,
 '总功率因数': 0.109,
 '瞬时总有功功率': 0.0099}

 ==================== 000102866265 ====================
{'A相电压': 239.5,
 'A相电压x电流': '143.70',
 'A相电流': 0.6,
 '上月组合有功总电能': 19483.45,
 '当前-上月': '432.54',
 '当前组合有功总电能': 19915.99,
 '总功率因数': 0.725,
 '瞬时总有功功率': 0.1049}


In [5]:
chn.close()